# Data & Indexing

## 1. Data Quality: The Foundation of RAG Systems

Before diving into chunking strategies, it's crucial to understand that **clean, well-structured data is the foundation** of any successful RAG (Retrieval-Augmented Generation) system.

---

### 📊 Data Sources

Your data can come from anywhere - the principles remain the same:

**Common Data Sources:**
- 📝 **Logs** - Application logs, system logs, audit trails
  - *Example*: Server error logs, user activity logs, API request logs
- 📄 **PDFs** - Documents, reports, manuals, research papers
  - *Example*: Product documentation, scientific papers, legal contracts
- 🗄️ **Document Stores** - Confluence, SharePoint, Google Docs
  - *Example*: Internal wikis, company policies, team documentation
- 🛠️ **Tools & APIs** - Jira, Slack, email archives
  - *Example*: Support tickets, team conversations, customer emails
- 💾 **Actual Databases** - SQL databases, NoSQL stores, data warehouses
  - *Example*: Customer records, product catalogs, transaction histories

---

### ⚠️ Why Data Quality Matters

**Garbage In = Garbage Out**: No matter how sophisticated your chunking strategy, embedding model, or vector database is, poor quality input data will lead to poor retrieval results and ultimately, poor AI responses.

**Real-world Impact:**
- **Noisy data** (e.g., HTML tags, formatting artifacts) → Embeddings capture irrelevant features → Wrong chunks retrieved
- **Inconsistent formatting** (e.g., "01/12/2024" vs "2024-01-12") → Model can't match related information
- **Encoding errors** (e.g., "donâ€™t" instead of "don't") → Semantic meaning lost → Poor search results
- **Duplicate content** → Wastes vector storage space and returns redundant results

**Example**: If your PDF has "Prïce: $100" (bad OCR) instead of "Price: $100", a user searching for "product price" may not find this information because the embedding won't recognize "Prïce" as related to "price".

---

### 🎯 Key Data Quality Principles

#### 1. **Remove Noise**

- Strip out unnecessary HTML tags, special characters
- Remove duplicate content, headers/footers that repeat
- **Clean up OCR errors from scanned documents:**

  **Common OCR Problems:**
  - Misrecognized characters (e.g., '0' mistaken for 'O', '1' for 'l', 'rn' for 'm')
  - Random symbols and artifacts (stray punctuation, page numbers in text)
  - Incorrectly split words across lines
  - Lost paragraph structure and formatting

  **How to Fix OCR Errors:**
  1. **Use spell checkers** - Tools like `textblob`, `pyspellchecker` to catch misrecognized words
  2. **Find/Replace patterns** - Use regex to fix common OCR mistakes:
     ```python
     # Example: Replace common OCR character errors
     text = text.replace('0', 'O').replace('rn', 'm')
     ```

#### 2. **Ensure Consistency**

- **Standardize date formats, units, terminology**
  - Dates: "2024-01-15" vs "Jan 15, 2024" vs "15/01/24" → Choose one format
  - Units: "5kg" vs "5 kilograms" vs "5000g" → Normalize to consistent unit
  - Terminology: "customer" vs "client" vs "user" → Use consistent term
  
- **Fix spelling and grammatical errors**
  - Use spell checkers: `textblob`, `pyspellchecker`, or language-specific tools
  - Fix common typos that could confuse semantic search
  
- **Normalize text encoding (UTF-8)**
  - **Why**: Different encodings (Latin-1, Windows-1252, UTF-16) represent characters differently
  - **Problem**: "café" in Latin-1 may appear as "cafÃ©" when read as UTF-8 (mojibake)
  - **Solution**: Detect source encoding and convert everything to UTF-8
  - **Tools**: 
    - `chardet` library to detect encoding
    - `ftfy` library to fix encoding errors
    - `unicodedata.normalize()` for Unicode normalization (NFC/NFD/NFKC/NFKD)

#### 3. **Preserve Structure**

- Maintain logical document hierarchy (headings, sections)
- Keep metadata (title, author, date, source)
- Preserve context indicators (lists, tables, code blocks)

---

1 hour of data cleaning can prevent 10+ hours of troubleshooting why your RAG system returns irrelevant answers.

---

Now let's proceed with chunking strategies on clean data...

## 2. Character-Based Chunking: Overview

Character-based chunking splits text based on character count, using various strategies to find natural boundaries (paragraphs, sentences, words).

We'll compare **3 LangChain approaches**:
1. **RecursiveCharacterTextSplitter** - Multi-separator intelligence (RECOMMENDED)
2. **CharacterTextSplitter** - Simple single separator
3. **NLTKTextSplitter** - Sentence-aware splitting

In [ ]:
# ============================================================================
# APPROACH 1: RecursiveCharacterTextSplitter (RECOMMENDED)
# 🎯 NOTE: Same splitter with chunk_size=50 → words, 300 → sentences, 1000 → paragraphs!
# ============================================================================

print("=" * 80)
print("1️⃣ RecursiveCharacterTextSplitter - Multi-separator intelligence")
print("=" * 80)

# --- Boundary logic for RecursiveCharacterTextSplitter ---
#   - Tries to split at the most meaningful separator first 
#     (paragraph, then newline, then sentence, then space, then character)
#   - Works down the list of separators, always ensuring the chunk does 
#     not exceed chunk_size
#   - If a clean boundary (like a paragraph or sentence) would make the 
#     chunk too big, it tries the next separator
#   - chunk_size is a strict upper limit: no chunk exceeds this value 
#     unless a single text unit (e.g., a paragraph) is longer than 
#     chunk_size (then it's split at the next available separator or 
#     hard limit)
#   - If the separators list is not given or is empty, the splitter will 
#     default to splitting at every character (i.e., no semantic boundaries, 
#     just raw character chunks of chunk_size)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from sample_data import SAMPLE_TEXT  # Import here for independent execution

# Test parameters
test_chunk_size = 100  # 🎯 This controls what we get!
test_overlap = 20

print(f"\n📝 Test text: {len(SAMPLE_TEXT)} characters\n")

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=test_chunk_size,
    chunk_overlap=test_overlap,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Tries these in order
)

recursive_chunks = recursive_splitter.split_text(SAMPLE_TEXT)
print(f"✅ Created {len(recursive_chunks)} chunks")
print("\n📊 How it works: Tries multiple separators in priority order")
print("   Pros: Preserves semantic boundaries, intelligent splitting")
print("   Cons: None - this is the recommended approach\n")

print("First 4 chunks:")
for i, chunk in enumerate(recursive_chunks[:6], 1):
    print(f"\nChunk {i} ({len(chunk)} chars):")
    print(f"'{chunk[:200]}{'...' if len(chunk) > 200 else ''}'")

print("\n" + "=" * 80)

In [ ]:
# ============================================================================
# APPROACH 2: CharacterTextSplitter - Simple single separator
# ============================================================================

print("=" * 80)
print("2️⃣ CharacterTextSplitter - Single separator approach")
print("=" * 80)

# --- Boundary logic for CharacterTextSplitter ---
#   - Splits only at the specified separator (e.g., space or newline)
#   - Builds chunks by adding text segments until the NEXT separator would exceed chunk_size
#   - When adding the next segment would go over the limit, it:
#     → Finishes the current chunk at the LAST separator that fit
#     → Starts a new chunk from that point
#   - Example: chunk_size=100, separator=" "
#     → Text: "word1 word2 word3 word4..." (each word ~25 chars)
#     → Adds: "word1 word2 word3 word4" (100 chars) ✓ Stops here (next word would exceed 100)
#     → Next separator (space before word5) is AFTER position 100, so chunk ends at word4
#
# ⚠️ CRITICAL: chunk_size CAN be exceeded (but only in rare cases):
#   - If a SINGLE segment between separators is longer than chunk_size
#   - Example: chunk_size=100, separator=" ", text has a 150-character 
#     URL or long word (no spaces)
#   - Result: That 150-char segment becomes its own chunk, EXCEEDING 
#     the 100-char limit
#   - Why? The splitter can't break it without finding a separator
#
#   - If separator is empty string, will apply the chunk size as hard limit.

from langchain_text_splitters import CharacterTextSplitter
from sample_data import SAMPLE_TEXT  # Import here for independent execution

# Test parameters (same as RecursiveCharacterTextSplitter for comparison)
test_chunk_size = 100
test_overlap = 20

print(f"\n📝 Test text: {len(SAMPLE_TEXT)} characters\n")

character_splitter = CharacterTextSplitter(
    separator=" ",  # Split at spaces
    chunk_size=test_chunk_size,
    chunk_overlap=test_overlap,
    length_function=len
)

character_chunks = character_splitter.split_text(SAMPLE_TEXT)
print(f"✅ Created {len(character_chunks)} chunks")
print("\n📊 How it works: Splits only at the specified separator (space)")
print("   Pros: Simple, predictable")
print("   Cons: May split in the middle of sentences or paragraphs")
print("   Note: All chunks ≤100 chars because SAMPLE_TEXT has no words >100 chars!\n")

print("First 4 chunks:")
for i, chunk in enumerate(character_chunks[:6], 1):
    print(f"\nChunk {i} ({len(chunk)} chars):")
    print(f"'{chunk[:200]}{'...' if len(chunk) > 200 else ''}'")

# 🧪 DEMONSTRATION: What happens with a segment longer than chunk_size?
print("\n" + "-" * 80)
print("🧪 EDGE CASE DEMO: Text with a 120-character 'word' (URL)")
print("-" * 80)

# Create test text with a long URL (no spaces, 120+ chars)
edge_case_text = "Normal text here. https://www.example-website-with-a-very-long-domain-name-that-exceeds-one-hundred-characters-to-demonstrate-the-edge-case.com/path More normal text after."

edge_splitter = CharacterTextSplitter(separator=" ", chunk_size=100, chunk_overlap=0, length_function=len)
edge_chunks = edge_splitter.split_text(edge_case_text)

print(f"\nCreated {len(edge_chunks)} chunks from edge case text:")
for i, chunk in enumerate(edge_chunks, 1):
    exceeds = "⚠️ EXCEEDS LIMIT!" if len(chunk) > 100 else "✓ Within limit"
    print(f"\nChunk {i}: {len(chunk)} chars {exceeds}")
    print(f"'{chunk}'")

print("\n" + "=" * 80)

In [ ]:
# ============================================================================
# APPROACH 3: NLTKTextSplitter - Sentence-aware splitting
# ============================================================================

print("=" * 80)
print("3️⃣ NLTKTextSplitter - Sentence-aware approach")
print("=" * 80)

# --- Boundary logic for NLTKTextSplitter ---
#   - Splits at sentence boundaries using NLTK's sentence tokenizer
#   - Merges sentences into a chunk until adding another would exceed 
#     chunk_size
#   - If a single sentence is longer than chunk_size, it will be its own 
#     chunk (and may greatly exceed chunk_size; this is common if your 
#     text has long sentences)
#   - The chunk_size is a soft limit: it will not break up sentences to 
#     enforce the limit

from langchain_text_splitters import NLTKTextSplitter
from sample_data import SAMPLE_TEXT  # Import here for independent execution

# Test parameters (same as others for comparison)
test_chunk_size = 100
test_overlap = 20

print(f"\n📝 Test text: {len(SAMPLE_TEXT)} characters\n")

nltk_splitter = NLTKTextSplitter(
    chunk_size=test_chunk_size,
    chunk_overlap=test_overlap,
    length_function=len
)

nltk_chunks = nltk_splitter.split_text(SAMPLE_TEXT)
print("\n📊 How it works: Splits at sentence boundaries using NLTK")
print("   Pros: Respects sentence boundaries, good for natural language")
print("   Cons: Sentences longer than chunk_size will be their own chunk,")
print("         so chunk_size is not a hard limit\n")

print("First 4 chunks:")
for i, chunk in enumerate(nltk_chunks[:4], 1):
    print(f"\nChunk {i} ({len(chunk)} chars):")
    print(f"'{chunk[:200]}{'...' if len(chunk) > 200 else ''}'")

print("\n" + "=" * 80)
print("💡 COMPARISON SUMMARY:")
print("   • RecursiveCharacterTextSplitter: Best balance (RECOMMENDED)")
print("   • CharacterTextSplitter: Simplest, but may break semantic units")
print("   • NLTKTextSplitter: Best for preserving complete sentences,")
print("                       but chunk_size is a soft limit")
print("=" * 80)

## 7. Semantic/Topic-Based Chunking

True semantic chunking that groups sentences based on **semantic similarity** rather than just paragraph boundaries.

**How it works:**
1. Splits text into sentences
2. Calculates semantic similarity between consecutive sentences using TF-IDF vectors
3. Creates chunk boundaries where similarity drops below a threshold
4. Adds configurable overlap to preserve context

**Term Frequency-Inverse Document Frequency**, is a numerical statistic used in information retrieval and text mining to evaluate the importance of a word in a document within a larger collection of documents.
    
**Why overlap matters:** Retrieved chunks may need context from semantically related preceding content.

In [ ]:
# Import and download NLTK only where needed (for semantic chunking)
import nltk
nltk.download('punkt', quiet=True)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def chunk_by_semantic_similarity(text: str, similarity_threshold: float = 0.5, overlap_sentences: int = 2, min_chunk_size: int = 2) -> list:
    """
    Semantic chunking based on sentence similarity using TF-IDF vectors
    
    Parameters:
    - similarity_threshold: Cosine similarity threshold (0-1). Lower = more chunks
    - overlap_sentences: Number of sentences from previous chunk to include as context
    - min_chunk_size: Minimum number of sentences per chunk
    
    Groups semantically similar sentences together and creates boundaries where
    the semantic topic shifts (similarity drops below threshold).
    
    Note: TF-IDF stands for Term Frequency-Inverse Document Frequency, a statistical measure used to evaluate how important a word is to a sentence or document.
    """
    # STEP 1: Split the text into individual sentences
    sentences = nltk.sent_tokenize(text)
    if len(sentences) <= min_chunk_size:
        return [text]
    
    # STEP 2: Convert sentences to numerical vectors using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        sentence_vectors = vectorizer.fit_transform(sentences)
    except ValueError:
        return [text]
    
    # STEP 3: Calculate similarity between consecutive sentences
    # similarities contains the cosine similarity between each pair of consecutive sentences 
    # (i and i+1) in the text.
    # Bigger cosine values (closer to 1) mean lower angle and hence more similar ; 
    # smaller cosine values (closer to 0) mean bigger angle and hence less similar.
    # Cosine 0=1
    # Cosine 90 = 0
    # Cosine 180 = -1
    similarities = [cosine_similarity(sentence_vectors[i:i+1], sentence_vectors[i+1:i+2])[0][0] for i in range(len(sentences) - 1)]
    
    # STEP 4: Identify chunk boundaries where topics change
    # The similarities list contains similarity scores between consecutive sentences:
    # similarities[i] = similarity between sentence[i] and sentence[i+1]
    # 
    # When similarity[i] < threshold, it means sentence[i] and sentence[i+1] are 
    # dissimilar (topic shift), so we create a boundary BETWEEN them.
    # 
    # Logic:
    # - Start with sentence 0 in chunk 0
    # - For each similarity score, check if it's below threshold
    # - If yes AND we have enough sentences in current chunk (>= min_chunk_size):
    #   → Create new chunk starting at sentence[i+1]
    #   → Reset counter to 1 (because sentence[i+1] is first in new chunk)
    # - Otherwise, keep adding sentences to current chunk
    chunk_boundaries = [0]  # First chunk starts at sentence 0
    current_chunk_size = 1  # We start with 1 sentence in the chunk
    
    for i, sim in enumerate(similarities):
        # At this point, we've seen i+1 sentences (0 to i inclusive)
        # similarities[i] tells us if sentence[i] and sentence[i+1] are similar
        
        if sim < similarity_threshold and current_chunk_size >= min_chunk_size:
            # Topic shift detected! Start new chunk at sentence i+1
            chunk_boundaries.append(i + 1)
            current_chunk_size = 1  # New chunk starts with 1 sentence
        else:
            # No topic shift, continue adding to current chunk
            current_chunk_size += 1
    
    # Ensure last boundary is at the end
    if chunk_boundaries[-1] != len(sentences):
        chunk_boundaries.append(len(sentences))
    
    # STEP 5: Create actual text chunks with optional overlap
    # chunk_boundaries contains indices like [0, 5, 10, 15] meaning:
    # - Chunk 0: sentences[0:5] (sentences 0,1,2,3,4)
    # - Chunk 1: sentences[5:10] (sentences 5,6,7,8,9)
    # - Chunk 2: sentences[10:15] (sentences 10,11,12,13,14)
    #
    # With overlap_sentences=2:
    # - Chunk 0: sentences[0:5] (no overlap for first chunk)
    # - Chunk 1: sentences[3:10] (includes 2 sentences from previous chunk)
    # - Chunk 2: sentences[8:15] (includes 2 sentences from previous chunk)
    #
    # This overlap preserves context across chunk boundaries!
    
    chunks = []
    for i in range(len(chunk_boundaries) - 1):
        start_idx = chunk_boundaries[i]
        end_idx = chunk_boundaries[i + 1]
        
        # Add overlap from previous chunk (except for first chunk)
        if i > 0 and overlap_sentences > 0:
            overlap_start = max(0, start_idx - overlap_sentences)
            chunk_sentences = sentences[overlap_start:end_idx]
        else:
            # First chunk has no previous context to overlap
            chunk_sentences = sentences[start_idx:end_idx]
        
        chunk = " ".join(chunk_sentences)
        chunks.append(chunk)
    
    return chunks

# Test semantic chunking
from sample_data import SAMPLE_TEXT  # Import here for independent execution

print("=" * 80)
print("SEMANTIC CHUNKING DEMONSTRATION")
print("=" * 80)

print("\n1. Semantic chunking with HIGH threshold (0.7) - more, smaller chunks:")
print("   (Creates new chunk even for moderate topic shifts; more sensitive, so more chunks)")
semantic_chunks_high = chunk_by_semantic_similarity(SAMPLE_TEXT, similarity_threshold=0.10, overlap_sentences=2)
print(f"✅ Created {len(semantic_chunks_high)} chunks\n")
for i, chunk in enumerate(semantic_chunks_high[:2]):
    print(f"Chunk {i+1} (length: {len(chunk)} chars):")
    print(f"{chunk[:250]}...\n")

print("\n2. Semantic chunking with LOW threshold (0.3) - fewer, larger chunks:")
print("   (Only creates new chunk when sentences are very different; less sensitive, so fewer chunks)")
semantic_chunks_low = chunk_by_semantic_similarity(SAMPLE_TEXT, similarity_threshold=0.3, overlap_sentences=2)
print(f"✅ Created {len(semantic_chunks_low)} chunks\n")
for i, chunk in enumerate(semantic_chunks_low[:2]):
    print(f"Chunk {i+1} (length: {len(chunk)} chars):")
    print(f"{chunk[:250]}...\n")

print("=" * 80)
print("Key Insight: Lower threshold = less sensitive to topic changes = fewer, larger chunks")
print("Overlap preserves semantic context across chunk boundaries!")
print("=" * 80)

## 7.0 Token-Based Chunking

Token-based chunking splits text into chunks based on the number of tokens (not characters). This is especially important for LLMs, as most models have token limits and count tokens differently than characters or words.

**What are tokens**
Tokens are not always words; they can be word pieces or punctuation.
LLMs process and count input/output in tokens, not characters or words.
Token limits (e.g., 4096 tokens for GPT-3.5) determine how much text the model can handle at once.
Example:
"cat" = 1 token
"unbelievable" = 3 tokens ("un", "believ", "able")
"Hello, world!" = 4 tokens ("Hello", ",", "world", "!")

**Key points:**
- A token is a unit of text as defined by the tokenizer (e.g., a word, part of a word, or punctuation) this is LLM specific.
- Token-based chunking ensures each chunk fits within the model's context window. (In Augmentation Layer for cost optimization)
- Common tokenizers: `tiktoken` (OpenAI), Hugging Face tokenizers, etc.
- Useful for preparing data for LLMs like GPT-3/4, which have strict token limits.

Below is a demonstration using the `tiktoken` tokenizer (used by OpenAI models).

In [ ]:
# TOKEN-BASED CHUNKING DEMONSTRATION (using LangChain's TokenTextSplitter)
# ============================================================================
# 🎯 KEY DIFFERENCE: chunk_size is in TOKENS, not characters!
# ============================================================================

try:
    from langchain_text_splitters import TokenTextSplitter
except ImportError:
    print("❌ langchain_text_splitters not installed. Run: pip install langchain-text-splitters")
    TokenTextSplitter = None

# --- Boundary logic for token-based chunking (LangChain) ---
#   - Splits text so that each chunk contains at most `chunk_size` tokens 
#     (as defined by the tokenizer)
#   - Overlap can be specified in tokens, not characters
#   - Uses tiktoken under the hood for OpenAI models (encoding_name param)
#   - Ensures compatibility with LLM context windows 
#     (e.g., 4096 tokens for GPT-3.5, 8k/32k for GPT-4)
#   - If langchain_text_splitters is not available, this cell will not run
#
# 💡 WHY TOKEN-BASED?
#   - LLMs count input in tokens, not characters
#   - Guarantees chunks fit within model limits 
#     (e.g., GPT-3.5's 4096 token limit)
#   - Predictable API costs (OpenAI charges per token)
#   - Character count varies: 100 tokens might be 200-500 chars 
#     depending on word complexity
#
# 📊 TOKEN vs CHARACTER EXAMPLE:
#  A token is a statistical unit that the model learned to recognize as 
#  a common text pattern, regardless of whether it has standalone meaning. 
#  This is specific to LLM.
#   - "cat" = 1 token, 3 characters
#   - "unbelievable" = 3 tokens (un + believ + able), 12 characters
#   - Same character count can have different token counts!

# Example usage with LangChain's TokenTextSplitter
if TokenTextSplitter is not None:
    from sample_data import SAMPLE_TEXT  # Import here for independent execution
    
    # Configure token-based chunking parameters
    token_chunk_size = 100    # Maximum 100 TOKENS per chunk (not characters!)
    token_chunk_overlap = 20    # 20 TOKEN overlap between chunks (for context preservation)
    
    # Initialize the token splitter with tiktoken encoding for OpenAI models
    token_splitter = TokenTextSplitter(
        chunk_size=token_chunk_size,        # Limit: 100 tokens per chunk
        chunk_overlap=token_chunk_overlap,  # Overlap: 20 tokens between chunks
        model_name="gpt-3.5-turbo"          # Use GPT-3.5-turbo's tokenizer (tiktoken), OpenAI's official fast tokenizer library,
        # When you installed langchain-text-splitters, it automatically installed tiktoken as a dependency.
        # Note: Use model_name for model identifiers (gpt-3.5-turbo, gpt-4), 
        # or encoding_name for encoding identifiers (cl100k_base, p50k_base, r50k_base)
    )
    
    # Split the sample text into token-based chunks
    token_chunks = token_splitter.split_text(SAMPLE_TEXT)
    
    # Display results
    print(f"✅ Created {len(token_chunks)} token-based chunks " +
          f"(chunk_size={token_chunk_size}, overlap={token_chunk_overlap})")
    print(f"   Note: Each chunk has ≤{token_chunk_size} tokens, " +
          "but character count varies!\n")
    
    # Show first 3 chunks with both character and token counts
    for i, chunk in enumerate(token_chunks[:6], 1):
        char_count = len(chunk)
        token_count = len(token_splitter._tokenizer.encode(chunk))
        print(f"\nToken Chunk {i}:")
        print(f"  - Character count: {char_count} chars")
        print(f"  - Token count: {token_count} tokens (≤{token_chunk_size})")
        print(f"  - Content preview: {chunk[:200]}...")
        
    print("\n" + "=" * 80)
    print("💡 OBSERVATION: Character count varies, but token count is " +
          "controlled!")
    print("   This ensures chunks always fit within LLM context windows.")
    print("=" * 80)

## 7.1 Store Chunks in Vector Database

Now let's take the semantic chunks and store them in a **vector store** with embeddings.
**Steps:**
1. Chunk the text (already done above)
2. Generate embeddings using an embedding model
3. Store embeddings in a vector database (ChromaDB)
4. Query to retrieve similar chunks

**Note:** This requires `chromadb` and `sentence-transformers` packages.

In [ ]:
# First, let's check if required packages are installed
try:
    import chromadb
    from sentence_transformers import SentenceTransformer
    print("✅ Required packages already installed!")
except ImportError as e:
    print("❌ Missing packages. Installing...")
    print("Run: pip install chromadb sentence-transformers")
    print(f"Error: {e}")

# Import dependencies for independent execution
from sample_data import SAMPLE_TEXT
# Note: chunk_by_semantic_similarity function must be defined before running this cell
# (either by running the semantic chunking cell first, or by importing from a module)

# Use the semantic chunks we created earlier with threshold=0.5
chunks_to_embed = chunk_by_semantic_similarity(SAMPLE_TEXT, similarity_threshold=0.15, overlap_sentences=2)
print(f"\n📦 Using {len(chunks_to_embed)} semantic chunks for embedding")

# STEP 1: Initialize embedding model
# 📊 DIMENSIONALITY: Different models produce different vector sizes
# Popular embedding models and their dimensions:
# - all-MiniLM-L6-v2: 384d (FAST, lightweight, good for most cases) ⚡ [USING THIS]
# - all-mpnet-base-v2: 768d (Better quality, more compute)
# - sentence-t5-base: 768d (T5-based, good for semantic search)
# - paraphrase-multilingual: 768d (Supports 50+ languages)
# - text-embedding-ada-002 (OpenAI): 1536d (Best quality, API cost)
# - text-embedding-3-small (OpenAI): 1536d (Improved ada-002)
# - text-embedding-3-large (OpenAI): 3072d (Highest quality, expensive)
# - voyage-large-2 (Voyage AI): 1536d (High-quality commercial API)
# - cohere-embed-v3 (Cohere): 1024d (Multilingual, commercial)
# 
# Trade-off: Higher dimensions = better semantic capture but slower search & more storage
print("\n🔄 Loading embedding model (this may take a moment)...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dimensional embeddings
print("✅ Embedding model loaded!")

# STEP 2: Generate embeddings for all chunks
# Each chunk becomes a 384-dimensional vector representing its meaning
print(f"\n🔄 Generating embeddings for {len(chunks_to_embed)} chunks...")
embeddings = embedding_model.encode(chunks_to_embed, show_progress_bar=True)
print(f"✅ Generated {len(embeddings)} embeddings, each with {embeddings[0].shape[0]} dimensions")

# STEP 3: Initialize ChromaDB (vector store)
# ChromaDB stores embeddings persistently and enables fast similarity search
print("\n🔄 Initializing ChromaDB vector store...")
chroma_client = chromadb.Client()  # In-memory database (for demo)

# 📐 SIMILARITY METRIC FOR VECTOR SEARCH: How we measure "closeness" between embeddings
# This is CRITICAL for retrieval quality - different from chunking similarity!
#
# Available options in ChromaDB: {"hnsw:space": "cosine" | "l2" | "ip"}
#
# 1. COSINE SIMILARITY (RECOMMENDED) ✓ [USING THIS]
#    - Measures: Angle between vectors (0° = identical, 90° = unrelated, 180° = opposite)
#    - Range: -1 to 1 (for embeddings, typically 0 to 1)
#    - Formula: dot(A,B) / (||A|| * ||B||)
#    - When to use: DEFAULT choice for semantic search with embeddings
#    - Why best: Direction matters, not magnitude - normalized automatically
#    - Best for: RAG, semantic search, Q&A systems, document similarity
#
# 2. L2 (EUCLIDEAN DISTANCE)
#    - Measures: Straight-line distance in n-dimensional space
#    - Range: 0 to infinity (0 = identical, larger = more different)
#    - Formula: sqrt(sum((A[i] - B[i])²))
#    - When to use: When vector magnitudes are meaningful and normalized
#    - Caveat: Sensitive to scale - embeddings with larger magnitudes rank higher
#    - Best for: Image embeddings, when using normalized embeddings only
#
# 3. IP (INNER PRODUCT / DOT PRODUCT)
#    - Measures: Both angle AND magnitude (unnormalized cosine)
#    - Range: -infinity to infinity
#    - Formula: sum(A[i] * B[i])
#    - When to use: ONLY when embeddings are pre-normalized (||A|| = ||B|| = 1)
#    - Why faster: Skips division step from cosine calculation
#    - Performance: ~10-20% faster than cosine for large-scale systems
#    - Best for: Production systems with normalized embeddings (OpenAI, Cohere)
#    - Caveat: If not normalized, longer vectors artificially rank higher!
#
# 💡 KEY DECISION FACTORS:
#    - Embedding Model: Most modern models (sentence-transformers, OpenAI) are normalized → use COSINE or IP
#    - Scale: Millions of vectors? Use IP for speed (if normalized)
#    - Safety: Unsure if normalized? Use COSINE (always safe)
#    - Legacy systems: L2 if that's what you've always used (consistency matters)
#
# 🎯 RECOMMENDATION FOR RAG:
#    - Start with COSINE (most intuitive, always correct)
#    - Switch to IP only if: (1) embeddings are normalized, (2) need extra speed at scale
#
# 📊 PERFORMANCE COMPARISON (1M vectors):
#    - Cosine: ~5ms per query
#    - L2: ~5ms per query  
#    - IP: ~4ms per query (fastest, but needs normalized vectors)
#
collection = chroma_client.get_or_create_collection(
    name="hr_policy_chunks",
    metadata={
        "hnsw:space": "cosine",  # Using cosine similarity - safest & most interpretable
        "description": "Semantic chunks from HR Remote Work Policy"
    }
)
print("✅ ChromaDB collection created with COSINE similarity!")

# STEP 4: Add chunks with embeddings to vector store
# Each chunk is stored with:
# - id: unique identifier
# - embedding: the vector representation
# - document: the original text
# - metadata: additional info (chunk index, length, etc.)
print(f"\n🔄 Storing {len(chunks_to_embed)} chunks in vector database...")
collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks_to_embed))],  # Unique IDs
    embeddings=embeddings.tolist(),  # Vector representations
    documents=chunks_to_embed,  # original text for retrieval alongside embeddings
    metadatas=[{"chunk_index": i, "length": len(chunk)} for i, chunk in enumerate(chunks_to_embed)]
)
print("✅ All chunks stored in vector database!")

print("\n" + "=" * 80)
print("VECTOR STORE SUMMARY")
print("=" * 80)
print(f"• Total chunks stored: {collection.count()}")
print(f"• Embedding dimensions: {embeddings[0].shape[0]}")
print(f"• Embedding model: all-MiniLM-L6-v2 (sentence-transformers)")
print(f"• Vector store: ChromaDB (in-memory)")
print("=" * 80)

## 7.2 Query the Vector Store: Vector & Hybrid Search

Now let's demonstrate retrieval methods:
1. **Vector Search**: Pure semantic similarity using embeddings
2. **Hybrid Search**: Combines keyword matching (BM25) + vector similarity for best results

In [ ]:
# Test query: Ask about technology requirements
query = "What are the internet speed requirements for remote work?"

print("=" * 80)
print("SEMANTIC SEARCH & HYBRID RETRIEVAL DEMONSTRATION")
print("=" * 80)
print(f"\n🔍 Query: '{query}'")

# STEP 1: Convert query to embedding using the same model
query_embedding = embedding_model.encode([query])[0]
print(f"✅ Query converted to {len(query_embedding)}-dimensional vector")

# ============================================================================
# VECTOR SEARCH: Semantic similarity using embeddings
# ============================================================================
print("\n" + "-" * 80)
print("1️⃣ VECTOR SEARCH (Semantic Similarity)")
print("-" * 80)
# ChromaDB uses HNSW (Hierarchical Navigable Small World) index
# This enables sub-millisecond search across billions of vectors
results_vector = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=3  # Retrieve top 3 most relevant chunks
)

print("Top 3 results by semantic similarity:\n")
for i, (doc, metadata, distance) in enumerate(zip(
    results_vector['documents'][0], 
    results_vector['metadatas'][0],
    results_vector['distances'][0]
), 1):
    similarity_score = 1 - distance  # Convert distance to similarity
    print(f"Rank {i} | Similarity: {similarity_score:.4f}")
    print(f"Content: {doc[:150]}...")
    print()

print("-" * 80)
print("💡 Vector search finds semantically similar chunks using embeddings")
print("-" * 80)

In [ ]:
# ============================================================================
# HYBRID SEARCH: Keyword (BM25) + Vector Re-ranking
# ============================================================================
print("=" * 80)
print("2️⃣ HYBRID SEARCH (Keyword + Vector Re-ranking)")
print("=" * 80)
print("💡 Combines BM25 keyword matching with semantic vector search")
print("   Strategy: Vector search finds top candidates → BM25 re-ranks them\n")

try:
    from rank_bm25 import BM25Okapi
    import numpy as np
    
    # ========================================================================
    # IMPORTANT: We are NOT creating a second vector store here!
    # ========================================================================
    # BM25 is a KEYWORD-BASED ranking algorithm (like classic search engines).
    # It does NOT use vectors or embeddings. Instead, it:
    #   1. Builds an inverted index (word → documents mapping)
    #   2. Scores documents based on keyword frequency and rarity (TF-IDF-like)
    #   3. Returns relevance scores for keyword matches
    #
    # We're combining TWO different retrieval systems:
    #   • ChromaDB (vector store) → Semantic similarity via embeddings
    #   • BM25 (keyword index) → Exact keyword matching via statistical ranking
    #
    # This "hybrid" approach gives us BOTH semantic understanding AND exact matches!
    # ========================================================================
    
    # STEP 1: Create BM25 keyword index (NOT a vector store!)
    # This builds an inverted index for fast keyword-based retrieval
    tokenized_chunks = [chunk.lower().split() for chunk in chunks_to_embed]
    bm25 = BM25Okapi(tokenized_chunks)  # BM25 index created from same chunks
    
    # STEP 2: Get vector similarity scores from the earlier vector search
    # ========================================================================
    # Reuse results_vector from the earlier query (top 3 chunks)
    # ========================================================================
    # We already queried ChromaDB above and got the top 3 chunks with their
    # distances. Now we'll extract those and convert to similarity scores.
    # ========================================================================
    
    # Extract chunk indices and distances from results_vector
    top_chunk_ids = results_vector['ids'][0]
    top_distances = results_vector['distances'][0]
    
    # Convert to similarity scores (1 - distance) and map to chunk indices
    vector_scores_dict = {}
    for chunk_id, distance in zip(top_chunk_ids, top_distances):
        chunk_idx = int(chunk_id.split('_')[1])  # Extract index from "chunk_0", "chunk_1", etc.
        vector_scores_dict[chunk_idx] = 1 - distance  # Convert distance to similarity
    
    # STEP 3: Get BM25 keyword scores for the top 3 chunks only
    # BM25 scores how well each of the top 3 chunks matches the query keywords
    query_tokens = query.lower().split()
    
    # Get BM25 scores only for the top 3 chunks
    bm25_scores_dict = {}
    for chunk_idx in vector_scores_dict.keys():
        # Score this specific chunk
        chunk_tokens = tokenized_chunks[chunk_idx]
        # BM25 score for single document
        bm25_score = bm25.get_scores(query_tokens)[chunk_idx]
        bm25_scores_dict[chunk_idx] = bm25_score
    
    # ========================================================================
    # BM25 SCORE NORMALIZATION: Scale scores to 0-1 range for fair comparison
    # ========================================================================
    # WHY NORMALIZE?
    # - BM25 raw scores are unbounded (can be any positive number)
    # - Vector similarity scores are already in 0-1 range (from cosine similarity)
    # - We need BOTH scores on the same scale to combine them fairly
    #
    # HOW IT WORKS (Min-Max Normalization):
    # Formula: normalized = (score - min) / (max - min)
    # 
    # Example: If BM25 scores are [0.5, 2.0, 5.0]
    #   - min = 0.5, max = 5.0, range = 4.5
    #   - Chunk 0: (0.5 - 0.5) / 4.5 = 0.0    (worst match)
    #   - Chunk 1: (2.0 - 0.5) / 4.5 = 0.33   (medium match)
    #   - Chunk 2: (5.0 - 0.5) / 4.5 = 1.0    (best match)
    #
    # RESULT: All scores now range from 0.0 (worst) to 1.0 (best)
    #
    # The +1e-10 prevents division by zero if all scores are identical
    # ========================================================================
    bm25_scores_list = list(bm25_scores_dict.values())
    bm25_min = min(bm25_scores_list)
    bm25_max = max(bm25_scores_list)
    
    bm25_normalized_dict = {}
    for chunk_idx, score in bm25_scores_dict.items():
        bm25_normalized_dict[chunk_idx] = (score - bm25_min) / (bm25_max - bm25_min + 1e-10)
    
    # STEP 4: Combine both scores with weighted average
    # ========================================================================
    # α (alpha) controls the balance between semantic and keyword search
    # ========================================================================
    # α = 0.7 means: hybrid_score = 0.7 × vector_score + 0.3 × bm25_score
    #
    # Adjust α based on your use case:
    # - α = 0.0 → Pure keyword search (BM25 only)
    # - α = 0.3 → Favor keywords (good for exact term matching)
    # - α = 0.5 → Equal balance
    # - α = 0.7 → Favor semantics (good for conceptual queries) ← USING THIS
    # - α = 1.0 → Pure semantic search (vector only)
    # ========================================================================
    alpha = 0.7  # 70% vector, 30% keyword
    
    # Calculate hybrid scores for the top 3 chunks
    hybrid_scores_dict = {}
    for chunk_idx in vector_scores_dict.keys():
        vector_score = vector_scores_dict[chunk_idx]
        bm25_score = bm25_normalized_dict[chunk_idx]
        hybrid_scores_dict[chunk_idx] = alpha * vector_score + (1 - alpha) * bm25_score
    
    # STEP 5: Rank chunks by combined hybrid score
    # Sort the chunks by hybrid score (highest first)
    sorted_hybrid = sorted(hybrid_scores_dict.items(), key=lambda x: x[1], reverse=True)
    
    print(f"Top 3 hybrid results (α={alpha}: {int(alpha*100)}% vector, " +
          f"{int((1-alpha)*100)}% keyword):\n")
    for rank, (chunk_idx, hybrid_score) in enumerate(sorted_hybrid, 1):
        vector_score = vector_scores_dict[chunk_idx]
        bm25_score = bm25_normalized_dict[chunk_idx]
        print(f"Rank {rank} | Hybrid: {hybrid_score:.4f} " +
              f"(Vector: {vector_score:.4f}, Keyword: {bm25_score:.4f})")
        print(f"Content: {chunks_to_embed[chunk_idx][:150]}...")
        print()
    
    print("💡 Adjust α: <0.5 favors keywords (exact matches), " +
          ">0.5 favors semantics (meaning)")
    
except ImportError:
    print("⚠️  BM25 not installed. Run: pip install rank-bm25")
    print("   Showing vector search results only.")

print("\n" + "=" * 80)
print("KEY OBSERVATIONS:")
print("=" * 80)
print("• Vector search uses HNSW index for fast approximate search")
print("• BM25 provides exact keyword matching for precision")
print("• Hybrid search combines BOTH: semantic understanding + keyword relevance")
print("• This is the 'Retrieval' part of RAG (Retrieval-Augmented Generation)")
print("=" * 80)